## Creating CF and ACDD compliant NetCDF file
*Author:* Corrado Motta - corradomotta92@gmail.com

This notebook aims to show how to add and possibly extract descriptive and administrative metadata from NetCDF files using python.
With descriptive metadata we mean all metadata that relates to Discovery and Identification in the FAIR principles. They usually includes info such as _title_, _author_, _subjects_, _keywords_, _publisher_, and _urls_. Examples of standards are __DataCite__, __DublinCore__, __ISO 19115__. They are mainly domain agnostic. We also referred to them as _global metadata_ in the NetCDF context. On the other hand, administritive metadata are used to provide technical support for managing
data in a dataset. Such metadata are domain specific. In the context of NetCDF, those are the attributes appended to each of our variables.

Useful links:
* [Notebook on FAIR and NetCDF](https://notebooks.githubusercontent.com/view/ipynb?color_mode=auto&commit=7438c171a8bfd838a97b9b859c8d92e0f9f01750&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f676973742f686576677972742f39663666613837383035643938636637386532356537373138616532336636622f7261772f373433386331373161386266643833386139376239623835396338643932653066396630313735302f6372656174655f4e65744344465f43465f414344442e6970796e62&logged_in=false&nwo=hevgyrt%2F9f6fa87805d98cf78e25e7718ae23f6b&path=create_NetCDF_CF_ACDD.ipynb&repository_id=99453764&repository_type=Gist)
* [Attribute Convention for Data Discovery](https://wiki.esipfed.org/Attribute_Convention_for_Data_Discovery_1-3)
* [CF Standard name table](https://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html)

In this notebook we show how to make a FAIR-compliant NECTDF file from raw data. We use two conventions to reach this goal:
* __CF__: Climate and Forecast convention. Mainly used for setting standard names to variables in NETCDF files and standard metadata for variables and dimensions.
* __ACDD__: Attribute for Climate and Data Discovery. It can be used together with CF to populate the global attributes of a .nc file. Some opensource software already exist to read the global attributes and automatically generate Dublin Core or ISO 19115 descriptive metadata.

Eventually, we also discuss how to generate/extract a XML file containg all the metadata following the ISO19115 standard schema. However, this part is not completed yet. For more information check the website pages.

### Table of Contents
* [1. Read configuration file](#read_cf)
* [2. Import raw data](#import_data)
* [3. Create NetCDF file with global metadata](#create_nc)
* [4. Read a NetCDF file and print metadata](#read_nc)
* [5. Export global metadata to ISO 19115 XML format](#export_iso)

### 1. Read configuration file <a class="anchor" id="read_cf"></a>
First of all we import all the needed packages.

In [3]:
# add our own module to interact with the database
import sys
sys.path.append("fairdata")

# geopandas for plotting position
import geopandas as gpd
# to read netcdf file
import netCDF4 as nc
# numpy is used to work with n-dimensional arrays
import numpy as np
# os miscellaneous
import os
# work with table
import pandas as pd
# to make figures
import matplotlib
# to work on netcdf files
import xarray
# to read conf file
import configparser
# to work with path
import ntpath
# to add date
from datetime import date
# to generate ISO19115
#from bas_metadata_library.standards.iso_19115_2 import MetadataRecordConfigV2, MetadataRecord
# To interact with the database
from fairdata import metadataDB
# for iso format
from datetime import datetime
# for plots
import pandas_bokeh
# for regular expressions
import re

In [4]:
pip install pysondb-v2

Note: you may need to restart the kernel to use updated packages.


In [5]:
from pysondb import PysonDB

Here we set the paths and the names of the file to analyze and the folder where to store results:

In [6]:
filePath   = r"data/20220914_134036_swamp2_turning_circle_azimuth30_rpm1600_deltayaw20.csv"
resultPath = r"demo_results"
confPath   = r"conf/conf.ini"

Then we check that the paths are good

In [7]:
# Extract file name from the file path
filename = ntpath.split(filePath)[1].split(".")[0]

# print them 
if(filePath):
    print("Path to raw data: {0}. Extracted filename: {1}".format(filePath, filename))
else:
    print("Path not available")
if(resultPath):
    print("path to directory where to store data: {0}".format(resultPath))
else:
    print("Path not available")
    
# check if result directory exists otherwise create it
if(not os.path.exists(resultPath)):
    os.makedirs(resultPath)
    print("Created directory for storing results.")

Path to raw data: data/20220914_134036_swamp2_turning_circle_azimuth30_rpm1600_deltayaw20.csv. Extracted filename: 20220914_134036_swamp2_turning_circle_azimuth30_rpm1600_deltayaw20
path to directory where to store data: demo_results


We are now ready to parse the configuration file which contains the global metadata. The configuration file can be manually filled using the _confTemplate.ini_ file or automatically generated from the interface that controls the vehicle (https://github.com/CorradoMotta/ASV_interface).

In [8]:
# read generated file
read_config = configparser.ConfigParser()
read_config.read(confPath)
my_complete_dict = dict(read_config.items('mandatory_global_attributes'))
my_complete_dict = my_complete_dict | dict(read_config.items('optional_global_attributes'))

In [9]:
print("The following attributes will be added:")
cont = 0
key_list = []
for key, value in my_complete_dict.items():
    if(value):
        cont +=1
        my_complete_dict[key] = str(value).replace('"','')
        key_list.append(key)
        print(str(cont) + ".", key, '->', value)

The following attributes will be added:
1. keywords -> "unmanned marine vehicles,marine robotics,autonomous systems"
2. institution -> CNR-INM
3. platform -> SWAMP
4. title -> Naval maneuver test in Venice
5. conventions -> "ACDD-1.3,CF-1.6"
6. license -> Creative Commons
7. summary -> First test of naval maneuver in Venice. ZigZag and circle maneuvers were done using several azimuth and trhust values.
8. creator_name -> "Ferretti Roberta, Bibuli Marco, Motta Corrado"
9. product_version -> 1
10. creator_email -> "roberta.ferretti@cnr.it,marco.bibuli@cnr.it,corrado.motta@inm.cnr.it"
11. project -> INNOVAMARE
12. processing_level -> raw data


#### 1b. Checking global variables against JSON database

Now we want to check if the attributes specifed in the configuration file satisfies the minimum set of mandatory global metadata stored our JSON database (check the "database" notebook for more information). All JSON database are stored in the database folder and have fixed names. We get access to the database entries by using the class _metadataDB_ in our own module. You can find info regarding our module in the gitHub pages. Also, in python, you can simply check what a method does using the help function:

In [10]:
help(metadataDB.metadataDB.getById)

Help on function getById in module fairdata.metadataDB:

getById(self, id)
    Returns the database entry found for the required ID.
    
    Args:
        id (str) : The database ID.
    
    Returns:
        dict: The dictionary of the element. None if not present.



In [11]:
# Opening JSON file
global_db = metadataDB.metadataDB('database/global_metadata.json')

# Iterate over mandatory global metadata. When one is not present, stop and print it
for key, value in global_db.getAll().items():
    if(value['required'] and not value['auto']):
        if(value['ACDD'].lower() in key_list):
            print(value['ACDD'] + ".. found")
        else:
            print(value['ACDD'], "NOT found!\n\nPlease add a value for ",value['ACDD'])
            break

Database with name global_metadata.json already existing.                 All further operation will directly connect to it.
title.. found
summary.. found
keywords.. found
Conventions.. found
creator_name.. found
creator_email.. found
institution.. found
platform.. found
license.. found
product_version.. found


### 2. Import raw data <a class="anchor" id="import_data"></a>
Let's import the telemetry data using read table method of pandas. We give as input the path, the delimiter as a single space, and the header list. As you can read in the github pages, we have two rows for the header, one for the log names and the second one for the standard names.

In [12]:
data = pd.read_table(filePath, delimiter = ',', converters={('time', 'time'):str,('date', 'date'): str}, header=[0,1])

In [13]:
# show the data
data

,date,time,NGC_latitude[MicroStrain 3DM-GX5-35],NGC_longitude[MicroStrain 3DM-GX5-35],projection_x_coordinate,projection_y_coordinate,horizontalAccuracy,llhPositionValidFlags,roll,pitch,...,RR_thruster_speed,FR_thruster_speed,RL_thruster_force,FL_thruster_force,RR_thruster_force,FR_thruster_force,RL_thruster_current,FL_thruster_current,RR_thruster_current,FR_thruster_current
,date,time,latitude,longitude,projection_x_coordinate,projection_y_coordinate,horizontalAccuracy,llhPositionValidFlags,platform_roll,platform_pitch,...,thruster_speed,thruster_speed,thruster_force,thruster_force,thruster_force,thruster_force,thruster_current,thruster_current,thruster_current,thruster_current
0,20220914,134036.000,45.436845,12.355075,-55.01,29.16,1.642,1,3.71,2.88,...,1603.8,1563.3,11.60,10.51,10.51,9.99,3.28,3.22,2.86,3.31
1,20220914,134036.000,45.436845,12.355075,-55.01,29.16,1.642,1,3.88,2.42,...,1611.9,1563.3,11.71,10.51,10.62,9.99,3.23,3.21,2.86,3.31
2,20220914,134036.000,45.436845,12.355075,-55.01,29.16,1.642,1,4.24,1.92,...,1611.9,1563.3,11.60,10.51,10.62,9.99,3.20,3.21,2.86,3.31
3,20220914,134036.249,45.436842,12.355073,-55.34,29.01,1.639,1,4.43,1.52,...,1611.9,1563.3,11.60,10.51,10.62,9.99,3.21,3.20,2.87,3.32
4,20220914,134036.249,45.436842,12.355073,-55.34,29.01,1.639,1,4.59,1.26,...,1595.7,1555.2,11.27,10.41,10.41,9.89,3.06,3.14,2.81,3.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,20220914,134151.749,45.436912,12.354833,-47.56,10.29,1.513,1,2.79,2.57,...,1595.7,1620.0,12.17,11.05,10.41,10.73,3.43,3.44,2.85,3.40
760,20220914,134151.749,45.436912,12.354833,-47.56,10.29,1.513,1,3.09,2.57,...,1595.7,1620.0,12.17,11.05,10.41,10.73,3.44,3.43,2.85,3.40
761,20220914,134152.000,45.436915,12.354832,-47.23,10.22,1.511,1,3.54,2.85,...,1587.6,1620.0,12.40,11.05,10.30,10.73,3.44,3.44,2.89,3.43


Let's remove the second header and save the header tuples on a dedicated list

In [14]:
data_columns = data.columns

# Remove header
data = data.droplevel(1, axis=1)

It is possible to save the __source__ of the data (i.e. The instrument or the method used to get to the measured/calculated data) by adding the name of it to the variable, using the square brackets as special character.

For example, `longitude[MicroStrain 3DM-GX5-35]` indicates that the longitude data were obtained with a Microstrain GPS.

Therefore, we use regular expression to get the source attribute to use later on, and we modify the variable name on the original table.

In [15]:
source_dict = {}
ndc = [] # new name filtered
mrs = ['[',']'] # markers
data_cln = [] # new multiindex data columns
regexPattern = r".*\{0}(.*)\{1}".format(mrs[0],mrs[1])
for key in data_columns:
    cln = key[0]
    if all([x in cln for x in mrs]):
        clnu = cln.split("[")[0]
        source_dict[clnu] = re.match(regexPattern, cln).group(1)
        ndc.append(clnu)
        data_cln.append((clnu,key[1]))
    else:
        ndc.append(cln)
        data_cln.append((cln,key[1]))

In [16]:
source_dict

{'NGC_latitude': 'MicroStrain 3DM-GX5-35',
 'NGC_longitude': 'MicroStrain 3DM-GX5-35'}

let's update the data columns name

In [17]:
data.columns = ndc

We are ready to add the global metadata. We want to follow the existing convention. It is important to have the datetime in **ISO 8601** format. In order to do so in an automated way, the date and time fields in the telemetry shall have a fixed format. Right now, in our telemetry, we have:

- date : YYYYMMDD
- time : HHMMSS.ms

By knowing that, we can create a _datetime_ object in python, then we can convert it to ISO.

In [18]:
# join date and time
data["datetime"] = data["date"].astype(str)+ " " + data["time"].astype(str)
# create a datetime python object
data["datetime"] = pd.to_datetime(data['datetime'], format='%yy%m%d %H%M%S.%s', infer_datetime_format=True)

We are ready to add all global attributes to a data structure in python. So that we can use it later on to fill the NetCDF file.

In [19]:
my_complete_dict["time_coverage_start"] = data["datetime"].min().isoformat()
my_complete_dict["time_coverage_end"] =   data["datetime"].max().isoformat()
my_complete_dict["geospatial_lat_max"] =  data['NGC_latitude'].max()
my_complete_dict["geospatial_lat_min"] =  data['NGC_latitude'].min()
my_complete_dict["geospatial_lon_min"] =  data['NGC_longitude'].min()
my_complete_dict["geospatial_lon_max"] =  data['NGC_longitude'].max()
my_complete_dict["date_created"] = datetime.now().isoformat()
my_complete_dict["time_coverage_duration"] = (data["datetime"].max() - data["datetime"].min()).isoformat()
my_complete_dict["time_coverage_resolution"] = "milliseconds"

# add to keylist
key_list = []
for key, value in my_complete_dict.items():
    if(value):
        key_list.append(key)

Check if any mandatory and auto global attributes are not filled yet.

In [20]:
# iterate over mandatory global metadata. When one is not present, stop and print it
for key, value in global_db.getAll().items():
    if(value['required'] and value['auto']):
        if(value['ACDD'].lower() in key_list):
            print(value['ACDD'] + ".. found")
        else:
            print(value['ACDD'], "NOT found!\n\nPlease add a value for ",value['ACDD'])
            break

time_coverage_start.. found
time_coverage_end.. found
geospatial_lat_max.. found
geospatial_lat_min.. found
geospatial_lat_units NOT found!

Please add a value for  geospatial_lat_units


Let's show on the map the trail of our vehicle. We use pandas bockeh functionality with openstreetmap as background.

In [21]:
# Create geodataframe for maps
gdf = gpd.GeoDataFrame(data, crs="EPSG:4326", geometry=gpd.points_from_xy(data.longitude, data.latitude))

AttributeError: 'DataFrame' object has no attribute 'longitude'

In [22]:
# load bokeh
pandas_bokeh.output_notebook()
pd.set_option('plotting.backend', 'pandas_bokeh')

Loading BokehJS ...

In [23]:
geo_plot = gdf.plot_bokeh(
        title=filename,
        tile_provider_url=r"http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png",
        figsize=(900, 600),
        line_color="black"
    )

NameError: name 'gdf' is not defined

We can also plot some data to check if our results look like expected. Let's print the variable names

In [24]:
data.columns

Index(['date', 'time', 'NGC_latitude', 'NGC_longitude',
       'projection_x_coordinate', 'projection_y_coordinate',
       'horizontalAccuracy', 'llhPositionValidFlags', 'roll', 'pitch', 'yaw',
       'surge_velocity', 'sway_velocity', 'heave_velocity', 'speedAccuracy',
       'headingAccuracy', 'nedVelocityValidFlags', 'roll_rate', 'pitch_rate',
       'yaw_rate', 'surge_acceleration', 'sway_acceleration',
       'heave_acceleration', 'azimuth_angle_reference', 'RL_azimuth_angle',
       'FL_azimuth_angle', 'RR_azimuth_angle', 'FR_azimuth_angle',
       'thruster_speed_reference', 'RL_thruster_speed', 'FL_thruster_speed',
       'RR_thruster_speed', 'FR_thruster_speed', 'RL_thruster_force',
       'FL_thruster_force', 'RR_thruster_force', 'FR_thruster_force',
       'RL_thruster_current', 'FL_thruster_current', 'RR_thruster_current',
       'FR_thruster_current', 'datetime'],
      dtype='object')

We plot the thruster speeds

In [25]:
# data.plot_bokeh(x='datetime', y=['FL_thruster_speed','FR_thruster_speed','RL_thruster_speed','RR_thruster_speed'], title = filename, figsize=(30,12))
rpm_plot = data.plot(figsize=(1200, 600),x='datetime', y=['FL_thruster_speed','FR_thruster_speed','RL_thruster_speed','RR_thruster_speed'], title = filename)

In [26]:
# remove the geometry field, which was added by bokeh
if 'geometry' in data:
    data.drop('geometry', inplace=True, axis=1)

### 3. Create NetCDF file with global metadata <a class="anchor" id="create_nc"></a>
Now that we know the data looks OK, we can create a NetCDF out of them. We use the _xarray_ package to do that. First we create an xarray object from the pandas data:

In [27]:
xr = xarray.Dataset.from_dataframe(data)

In [28]:
xr

<xarray.Dataset>
Dimensions:                   (index: 764)
Coordinates:
  * index                     (index) int64 0 1 2 3 4 5 ... 759 760 761 762 763
Data variables: (12/42)
    date                      (index) object '20220914' ... '20220914'
    time                      (index) object '134036.000' ... '134152.249'
    NGC_latitude              (index) float64 45.44 45.44 45.44 ... 45.44 45.44
    NGC_longitude             (index) float64 12.36 12.36 12.36 ... 12.35 12.35
    projection_x_coordinate   (index) float64 -55.01 -55.01 ... -47.23 -46.9
    projection_y_coordinate   (index) float64 29.16 29.16 29.16 ... 10.22 10.22
    ...                        ...
    FR_thruster_force         (index) float64 9.99 9.99 9.99 ... 10.73 10.73
    RL_thruster_current       (index) float64 3.28 3.23 3.2 ... 3.44 3.43 3.4
    FL_thruster_current       (index) float64 3.22 3.21 3.21 ... 3.44 3.61 3.61
    RR_thruster_current       (index) float64 2.86 2.86 2.86 ... 2.89 2.89 2.89
    FR_thruster_current       (index) float64 3.31 3.31 3.31 ... 3.43 3.44 3.45
    datetime                  (index) datetime64[ns] 2022-09-14T13:40:36 ... ...

By printing it, we can already see that now it has assumed the format of a NETCDF file

It is time to add the __global__ metadata stored in our configuration file to the Netcdf

In [29]:
for key, value in my_complete_dict.items():
    if(value):
        xr.attrs[key] = value

In [30]:
# We can see the global attributes added!
xr

<xarray.Dataset>
Dimensions:                   (index: 764)
Coordinates:
  * index                     (index) int64 0 1 2 3 4 5 ... 759 760 761 762 763
Data variables: (12/42)
    date                      (index) object '20220914' ... '20220914'
    time                      (index) object '134036.000' ... '134152.249'
    NGC_latitude              (index) float64 45.44 45.44 45.44 ... 45.44 45.44
    NGC_longitude             (index) float64 12.36 12.36 12.36 ... 12.35 12.35
    projection_x_coordinate   (index) float64 -55.01 -55.01 ... -47.23 -46.9
    projection_y_coordinate   (index) float64 29.16 29.16 29.16 ... 10.22 10.22
    ...                        ...
    FR_thruster_force         (index) float64 9.99 9.99 9.99 ... 10.73 10.73
    RL_thruster_current       (index) float64 3.28 3.23 3.2 ... 3.44 3.43 3.4
    FL_thruster_current       (index) float64 3.22 3.21 3.21 ... 3.44 3.61 3.61
    RR_thruster_current       (index) float64 2.86 2.86 2.86 ... 2.89 2.89 2.89
    FR_thruster_current       (index) float64 3.31 3.31 3.31 ... 3.43 3.44 3.45
    datetime                  (index) datetime64[ns] 2022-09-14T13:40:36 ... ...
Attributes: (12/21)
    keywords:                  unmanned marine vehicles,marine robotics,auton...
    institution:               CNR-INM
    platform:                  SWAMP
    title:                     Naval maneuver test in Venice
    conventions:               ACDD-1.3,CF-1.6
    date_created:              2023-08-07T15:29:50.652765
    ...                        ...
    geospatial_lat_max:        45.436918
    geospatial_lat_min:        45.436819
    geospatial_lon_min:        12.354832
    geospatial_lon_max:        12.355077
    time_coverage_duration:    P0DT0H1M16.249S
    time_coverage_resolution:  milliseconds

Now, we can add the __attributes__ metadata to each variable to all variables that are found 

In [31]:
# Set json file

# Opening JSON file
variable_db = metadataDB.metadataDB('database/variable_metadata.json')

# get all data
variables = variable_db.getAll()

Database with name variable_metadata.json already existing.                 All further operation will directly connect to it.


In [32]:
# iterate over each variable in the table and look for it in the database.
for key in data_cln:
    attr = variable_db.getEntry('long_name', key[1])
    if(key[0] in source_dict): xr[key[0]].attrs["source"] = source_dict[key[0]]
    if(attr):
        print("Attributes found for variable", key[0])
        for attr_name, value in attr[0].items():
            if(attr_name!='version' and value):
                xr[key[0]].attrs[attr_name] = value
    else:
        print("Attributes NOT found for variable", key[0])
print("\nAll done!")

Attributes found for variable date
Attributes found for variable time
Attributes found for variable NGC_latitude
Attributes found for variable NGC_longitude
Attributes found for variable projection_x_coordinate
Attributes found for variable projection_y_coordinate
Attributes found for variable horizontalAccuracy
Attributes found for variable llhPositionValidFlags
Attributes found for variable roll
Attributes found for variable pitch
Attributes found for variable yaw
Attributes found for variable surge_velocity
Attributes found for variable sway_velocity
Attributes found for variable heave_velocity
Attributes found for variable speedAccuracy
Attributes found for variable headingAccuracy
Attributes found for variable nedVelocityValidFlags
Attributes found for variable roll_rate
Attributes found for variable pitch_rate
Attributes found for variable yaw_rate
Attributes found for variable surge_acceleration
Attributes found for variable sway_acceleration
Attributes found for variable heave_

In [33]:
# We can see the attributes on each variable now!
xr

<xarray.Dataset>
Dimensions:                   (index: 764)
Coordinates:
  * index                     (index) int64 0 1 2 3 4 5 ... 759 760 761 762 763
Data variables: (12/42)
    date                      (index) object '20220914' ... '20220914'
    time                      (index) object '134036.000' ... '134152.249'
    NGC_latitude              (index) float64 45.44 45.44 45.44 ... 45.44 45.44
    NGC_longitude             (index) float64 12.36 12.36 12.36 ... 12.35 12.35
    projection_x_coordinate   (index) float64 -55.01 -55.01 ... -47.23 -46.9
    projection_y_coordinate   (index) float64 29.16 29.16 29.16 ... 10.22 10.22
    ...                        ...
    FR_thruster_force         (index) float64 9.99 9.99 9.99 ... 10.73 10.73
    RL_thruster_current       (index) float64 3.28 3.23 3.2 ... 3.44 3.43 3.4
    FL_thruster_current       (index) float64 3.22 3.21 3.21 ... 3.44 3.61 3.61
    RR_thruster_current       (index) float64 2.86 2.86 2.86 ... 2.89 2.89 2.89
    FR_thruster_current       (index) float64 3.31 3.31 3.31 ... 3.43 3.44 3.45
    datetime                  (index) datetime64[ns] 2022-09-14T13:40:36 ... ...
Attributes: (12/21)
    keywords:                  unmanned marine vehicles,marine robotics,auton...
    institution:               CNR-INM
    platform:                  SWAMP
    title:                     Naval maneuver test in Venice
    conventions:               ACDD-1.3,CF-1.6
    date_created:              2023-08-07T15:29:50.652765
    ...                        ...
    geospatial_lat_max:        45.436918
    geospatial_lat_min:        45.436819
    geospatial_lon_min:        12.354832
    geospatial_lon_max:        12.355077
    time_coverage_duration:    P0DT0H1M16.249S
    time_coverage_resolution:  milliseconds

Now we can save it as a NETCDF with the following function:

In [41]:
# create a result path for it
result_path = os.path.join(resultPath, filename + ".nc")

# save to nc
xr.to_netcdf(result_path)
print("saved to {0}".format(result_path))

saved to demo_results\20220914_134036_swamp2_turning_circle_azimuth30_rpm1600_deltayaw20.nc


### 4. Read a NetCDF file and print metadata <a class="anchor" id="read_nc"></a>
To read the NetCDF we use the homonym python package

In [51]:
ds = nc.Dataset(result_path)
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    keywords: unmanned marine vehicles,marine robotics,autonomous systems
    institution: CNR-INM
    platform: SWAMP
    title: Naval maneuver test in Venice
    conventions: ACDD-1.3,CF-1.6
    date_created: 2023-08-07T15:29:50.652765
    license: Creative Commons
    summary: First test of naval maneuver in Venice. ZigZag and circle maneuvers were done using several azimuth and trhust values.
    creator_name: Ferretti Roberta, Bibuli Marco, Motta Corrado
    product_version: 1
    creator_email: roberta.ferretti@cnr.it,marco.bibuli@cnr.it,corrado.motta@inm.cnr.it
    project: INNOVAMARE
    processing_level: raw data
    time_coverage_start: 2022-09-14T13:40:36
    time_coverage_end: 2022-09-14T13:41:52.249000
    geospatial_lat_max: 45.436918
    geospatial_lat_min: 45.436819
    geospatial_lon_min: 12.354832
    geospatial_lon_max: 12.355077
    time_coverage_duration: P0DT0H1M16.249S
    time_

As an alternative we can use xarray as well

In [52]:
ds_disk = xarray.open_dataset(result_path)

In [53]:
ds_disk

<xarray.Dataset>
Dimensions:                   (index: 764)
Coordinates:
  * index                     (index) int64 0 1 2 3 4 5 ... 759 760 761 762 763
Data variables: (12/42)
    date                      (index) object ...
    time                      (index) object ...
    NGC_latitude              (index) float64 ...
    NGC_longitude             (index) float64 ...
    projection_x_coordinate   (index) float64 ...
    projection_y_coordinate   (index) float64 ...
    ...                        ...
    FR_thruster_force         (index) float64 ...
    RL_thruster_current       (index) float64 ...
    FL_thruster_current       (index) float64 ...
    RR_thruster_current       (index) float64 ...
    FR_thruster_current       (index) float64 ...
    datetime                  (index) datetime64[ns] ...
Attributes: (12/21)
    keywords:                  unmanned marine vehicles,marine robotics,auton...
    institution:               CNR-INM
    platform:                  SWAMP
    title:                     Naval maneuver test in Venice
    conventions:               ACDD-1.3,CF-1.6
    date_created:              2023-08-07T15:29:50.652765
    ...                        ...
    geospatial_lat_max:        45.436918
    geospatial_lat_min:        45.436819
    geospatial_lon_min:        12.354832
    geospatial_lon_max:        12.355077
    time_coverage_duration:    P0DT0H1M16.249S
    time_coverage_resolution:  milliseconds

Let's print the available global attributes

In [45]:
for key, value in ds_disk.attrs.items() :
    print(key + ": " + str(value))

keywords: unmanned marine vehicles,marine robotics,autonomous systems
institution: CNR-INM
platform: SWAMP
title: Naval maneuver test in Venice
conventions: ACDD-1.3,CF-1.6
date_created: 2023-08-07T15:29:50.652765
license: Creative Commons
summary: First test of naval maneuver in Venice. ZigZag and circle maneuvers were done using several azimuth and trhust values.
creator_name: Ferretti Roberta, Bibuli Marco, Motta Corrado
product_version: 1
creator_email: roberta.ferretti@cnr.it,marco.bibuli@cnr.it,corrado.motta@inm.cnr.it
project: INNOVAMARE
processing_level: raw data
time_coverage_start: 2022-09-14T13:40:36
time_coverage_end: 2022-09-14T13:41:52.249000
geospatial_lat_max: 45.436918
geospatial_lat_min: 45.436819
geospatial_lon_min: 12.354832
geospatial_lon_max: 12.355077
time_coverage_duration: P0DT0H1M16.249S
time_coverage_resolution: milliseconds


We can also define a simple function to read a single attribute on demand and one to return a variable object.

In [46]:
def getAttribute(myds, attribute):
    # return the value of the attribute given as argument
    
    my_attr = None
    try:
         my_attr = getattr(myds, attribute)
    except AttributeError as e: 
        print("arg <{0}> not present in the .nc file".format(attribute))
    return my_attr

def getVariable(myds, variable):
    # return the object of the variable given as argument
    return myds.variables.get(variable, None)
        

In [47]:
# geospatial_lat_max
my_attr = "summary"
print(getAttribute(ds, my_attr))

First test of naval maneuver in Venice. ZigZag and circle maneuvers were done using several azimuth and trhust values.


Let's check all variable names

In [48]:
list(ds.variables.keys())

['index',
 'date',
 'time',
 'NGC_latitude',
 'NGC_longitude',
 'projection_x_coordinate',
 'projection_y_coordinate',
 'horizontalAccuracy',
 'llhPositionValidFlags',
 'roll',
 'pitch',
 'yaw',
 'surge_velocity',
 'sway_velocity',
 'heave_velocity',
 'speedAccuracy',
 'headingAccuracy',
 'nedVelocityValidFlags',
 'roll_rate',
 'pitch_rate',
 'yaw_rate',
 'surge_acceleration',
 'sway_acceleration',
 'heave_acceleration',
 'azimuth_angle_reference',
 'RL_azimuth_angle',
 'FL_azimuth_angle',
 'RR_azimuth_angle',
 'FR_azimuth_angle',
 'thruster_speed_reference',
 'RL_thruster_speed',
 'FL_thruster_speed',
 'RR_thruster_speed',
 'FR_thruster_speed',
 'RL_thruster_force',
 'FL_thruster_force',
 'RR_thruster_force',
 'FR_thruster_force',
 'RL_thruster_current',
 'FL_thruster_current',
 'RR_thruster_current',
 'FR_thruster_current',
 'datetime']

And print a single variable

In [49]:
my_var = "FR_azimuth_angle"
obj_var = getVariable(ds, my_var)
print(obj_var)

<class 'netCDF4._netCDF4.Variable'>
float64 FR_azimuth_angle(index)
    _FillValue: nan
    long_name: azimuth_angle
    units: degree
    coverage_content_type: physicalMeasurement
    comment: Platform azimuth angle is the horizontal angle between the line of sight from the observation point to the platform and a reference direction at the observation point, which is often due north.
unlimited dimensions: 
current shape = (764,)
filling on


In [50]:
my_var = "sway_velocity"
obj_var = getVariable(ds, my_var)
obj_var

<class 'netCDF4._netCDF4.Variable'>
float64 sway_velocity(index)
    _FillValue: nan
    long_name: platform_sway_rate_starboard
    standard_name: platform_sway_rate_starboard
    units: m s-1
    coverage_content_type: auxiliaryInformation
    comment: Sway rate is the rate of displacement along an axis that is perpendicular to both the local vertical axis and the nominal forward motion direction of the platform. Typycally represented with the v letter.
unlimited dimensions: 
current shape = (764,)
filling on

### 4. Export global metadata to ISO 19115 XML format <a class="anchor" id="export_iso"></a>

The metadata set in the conf.ini file are then added to the NETCDF4 using ACDD. However, we also want to be able to generate a ISO 199115 compliant metadata file, in the XML format. In fact, this is required by many different online repositories. 

Mapping between ACDD and ISO 19115 is provided [here](https://wiki.esipfed.org/Attribute_Convention_for_Data_Discovery_Mappings).

__Note:__ this is under development. Right now, a python software packet named [bas-metadata-library](https://pypi.org/project/bas-metadata-library/) is used to generate the ISO file. This is done from the conf.ini list and not directly from the NETCDF4, which would be a more auspicable solution.

In [ ]:
# set contact
if(my_complete_dict["creator_name"]):
    individual_contact = [{"individual": {"name": my_complete_dict["creator_name"]}, "role": ["originator"]}]
else:
    print("not implemented yet")
    
    
minimal_record_config = {
    "hierarchy_level": "dataset",
    "metadata": {
        "language": "eng",
        "character_set": "utf-8",
        "contacts": individual_contact,
        "date_stamp": datetime.now(),
    },
    "identification": {
        "title": {"value": my_complete_dict['title']},
        "dates": {"creation": {"date": datetime.now(), "date_precision": "year"}},
        "abstract": my_complete_dict['summary'],
        "keywords":[{"terms":[{"term": item} for item in my_complete_dict['keywords'].split(",")]}],
        "character_set": "utf-8",
        "language": "eng",
        "topics": ["geoscientificInformation"],
        "extent": {
            "geographic": {
                "bounding_box": {
                    "west_longitude": my_complete_dict["geospatial_lon_min"],
                    "east_longitude": my_complete_dict["geospatial_lon_max"],
                    "south_latitude": my_complete_dict["geospatial_lat_min"],
                    "north_latitude": my_complete_dict["geospatial_lat_max"],
                }
            }
        },
    },
}
configuration = MetadataRecordConfigV2(**minimal_record_config)
record = MetadataRecord(configuration=configuration)
document = record.generate_xml_document()

# output document
result_path = os.path.join(resultPath, filename + "_metadata.xml")
# print(document.decode())
f = open(result_path, "w")
f.write(document.decode())
f.close()
print("metadata saved in", result_path)